In [55]:
# Import necessary libraries
import pandas as pd

# Upload the Excel file
from google.colab import files
uploaded = files.upload()

# Load the Excel file
file_path = list(uploaded.keys())[0]
excel_data = pd.ExcelFile(file_path)

# Load all necessary sheets into DataFrames
orders_df = excel_data.parse('Orders')
products_df = excel_data.parse('Products')
sales_reps_df = excel_data.parse('Sales_Reps')
regions_df = excel_data.parse('Regions')
customers_df = excel_data.parse('Customers')


Saving PulseShop Sales.xlsx to PulseShop Sales (9).xlsx


In [56]:
# Categorize time of day based on the hour
def categorize_time(hour):
    if 6 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 18:
        return "Afternoon"
    elif 18 <= hour < 24:
        return "Evening"
    else:
        return "Night"

# Apply categorization to determine the time of day
orders_df['Time_of_Day'] = orders_df['Hour'].apply(categorize_time)

# Convert Order_Date to datetime if not already
orders_df['Order_Date'] = pd.to_datetime(orders_df['Order_Date'])

# Aggregate total sales per month and calculate growth rates
orders_df['YearMonth'] = orders_df['Order_Date'].dt.to_period('M')
monthly_sales = orders_df.groupby('YearMonth')['Total_Sales'].sum().reset_index()
monthly_sales['MoM_Growth'] = monthly_sales['Total_Sales'].pct_change()

# Aggregate total sales per year and calculate YoY growth rates
orders_df['Year'] = orders_df['Order_Date'].dt.year
yearly_sales = orders_df.groupby('Year')['Total_Sales'].sum().reset_index()
yearly_sales['YoY_Growth'] = yearly_sales['Total_Sales'].pct_change()

# Merge growth rates back to the orders data
orders_df = pd.merge(
    orders_df,
    monthly_sales[['YearMonth', 'MoM_Growth']],
    left_on='YearMonth',
    right_on='YearMonth',
    how='left'
)
orders_df = pd.merge(
    orders_df,
    yearly_sales[['Year', 'YoY_Growth']],
    on='Year',
    how='left'
)

# Fill NaN values with 0
orders_df.fillna(0, inplace=True)


# Calculate Average Order Size
orders_df['Average_Order_Size'] = orders_df['Total_Sales'] / orders_df['Quantity_Sold']


In [57]:
# Calculate key performance indicators (KPIs)
kpis = {
    "Total Sales": orders_df['Total_Sales'].sum(),
    "Total Quantity Sold": orders_df['Quantity_Sold'].sum(),
    "Total Profit": orders_df['Profit'].sum(),
    "Average Order Size": orders_df['Average_Order_Size'].mean(),
    "Sales Growth Rate (MoM)": monthly_sales['MoM_Growth'].mean(skipna=True),
    "Sales Growth Rate (YoY)": yearly_sales['YoY_Growth'].mean(skipna=True)
}

print("Key Performance Indicators (KPIs):")
print(kpis)


Key Performance Indicators (KPIs):
{'Total Sales': 2248618.567, 'Total Quantity Sold': 10822, 'Total Profit': 477851.10022, 'Average Order Size': 208.24683750000003, 'Sales Growth Rate (MoM)': 0.19032539508795945, 'Sales Growth Rate (YoY)': 1.8188082504237086}


In [58]:
# Top products
top_products = orders_df.groupby('Product_ID')['Total_Sales'].sum().nlargest(2).reset_index()
top_products = pd.merge(top_products, products_df, on='Product_ID')

# Top regions
top_regions = orders_df.groupby('Region_ID')['Total_Sales'].sum().nlargest(2).reset_index()
top_regions = pd.merge(top_regions, regions_df, on='Region_ID')

# Top customers
top_customers = orders_df.groupby('Region_ID')['Total_Sales'].sum().nlargest(2).reset_index()

print("Top Products:")
print(top_products)
print("Top Regions:")
print(top_regions)
print("Top Customers:")
print(top_customers)


Top Products:
   Product_ID  Total_Sales     Product_Name    Product_Category
0         202   479660.974  Electric Kettle  Kitchen Appliances
1         204   452065.627     Air Purifier       Home & Living
Top Regions:
   Region_ID  Total_Sales Region_Name
0        105   467050.298        Arad
1        101   458744.325      Brașov
Top Customers:
   Region_ID  Total_Sales
0        105   467050.298
1        101   458744.325


In [59]:
# Save processed data to an xlsx file
# Read data from the original file (Excel file)
xls = pd.ExcelFile(file_path)

# Read different sheets from the file
products = pd.read_excel(xls, sheet_name='Products')
sales_reps = pd.read_excel(xls, sheet_name='Sales_Reps')
regions = pd.read_excel(xls, sheet_name='Regions')
customers = pd.read_excel(xls, sheet_name='Customers')

# Format the Hire_Date column in the Sales_Reps sheet
sales_reps['Hire_Date'] = pd.to_datetime(sales_reps['Hire_Date'], errors='coerce').dt.strftime('%m/%d/%Y')
orders_df['Order_Date'] = pd.to_datetime(orders_df['Order_Date'], errors='coerce').dt.strftime('%m/%d/%Y')

# Write a new Excel file containing all sheets
output_file_path = 'Processed_Sales_Data.xlsx'
with pd.ExcelWriter(output_file_path) as writer:
    orders_df.to_excel(writer, sheet_name='Orders', index=False)
    products.to_excel(writer, sheet_name='Products', index=False)
    sales_reps.to_excel(writer, sheet_name='Sales_Reps', index=False)
    regions.to_excel(writer, sheet_name='Regions', index=False)
    customers.to_excel(writer, sheet_name='Customers', index=False)

print(f"File saved successfully at: {output_file_path}")

from google.colab import files
files.download(output_file_path)

File saved successfully at: Processed_Sales_Data.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>